### Fake Data

In [1]:
# Creating fake data for linear regression
import numpy as np
import matplotlib.pyplot as plt

# Create a dataset
np.random.seed(0)
w_true, b_true = 15, 25
X = 10 * np.random.rand(100, 1)
y = w_true * X + b_true + 5 * np.random.randn(100, 1)

# Plot the data
plt.plot(X, y, "b.")
plt.xlabel("$x_1$", fontsize=18)
plt.ylabel("$y$", rotation=0, fontsize=18)
plt.axis([0, 10, 0, 100])
plt.show()

### Optimizer LLM

In [2]:
import ast
from tqdm import tqdm
from linReg_helpers import client

def opt_llm(text_score_pairs):
    user_prompt = """
        Now you will help me optimize a function with two input variables w, b. I have some (w,b) pairs
        and the function values at those points. The pairs are arranged in ascending order based on their
        function value categories.

        The equation of a line is y = w * x + b, where w is the slope and b is the y-intercept.
        The quality of the line can be placed into one of 5 categories:
        1. Excellent
        2. Good
        3. Fair
        4. Poor
        5. Very Poor

        ## Excellent
        A line that fits the data points very well and passes through as many points as possible while having the 
        lowest possible error rate between the points and the line should be labelled as "Excellent".

        ## Good
        A line that fits the data points well and passes through many of the points while having a small error rate
        between the points and the line should be labelled as "Good".

        ## Fair
        A line that fits the data points reasonably well and passes through some of the points while having a moderate
        error rate between the points and the line should be labelled as "Fair".

        ## Poor
        A line that fits the data points poorly and passes through few of the points while having a large error rate
        between the points and the line should be labelled as "Poor".

        ## Very Poor
        A line that fits the data points very poorly and passes through none of the points while having an incredibly 
        large error rate between the points and the line should be labelled as "Very Poor".


        {text_score_pairs}

        Give me new (w,b) pairs that are different from all pairs above, and has a function value category of "Excellent".
        Do not write code. The output must be a list of 10 unique (w,b) pairs, where w and b are
        numerical values. The (w,b) pairs can be positive or negative, and can have decimal points.

        Example: [(1, 2), (3, 4), (5, -6), (7, -8), (0.5, 10), (11, 12), (13, 0.9), (15, 16), (-17, 18), (19, -20)]

    """

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system", 
                "content": """
                You are an optimization expert. The user has some (w,b) pairs along their corresponding function values.
                Your task is to generate a list of 10 (w,b) pairs that will have a function value as low as possible.

                Example: [(1, 2), (3, 4), (5, -6), (7, -8), (0.5, 10), (11, 12), (13, 0.9), (15, 16), (-17, 18), (19, -20)]

                Do not print anything other than a list of 10 (w,b) pairs. Do not write a sentence. Do not write code.
                """
            },
            {
                "role": "user", 
                "content": user_prompt.format(text_score_pairs=text_score_pairs)
            },
        ],
        model="gpt-3.5-turbo-0125",
        max_tokens=4096,
        temperature=1,
        # response_format={ "type": "json_object" },
    )
    result = chat_completion.choices[0].message.content
    return result

output = opt_llm("""
input:
w=18, b=15
value:
10386334
        
input:
w=17, b=18
value:
9204724""")

tuple_list = ast.literal_eval(output)

output, tuple_list

('[(-2.5, 3.5), (0.8, 0.2), (4.6, -7.4), (6.2, -9.8), (-1.1, 11.5), (8.3, 13.1), (10.9, -21.7), (15.4, 16.8), (-17.2, 18.6), (19.7, -20.3)]',
 [(-2.5, 3.5),
  (0.8, 0.2),
  (4.6, -7.4),
  (6.2, -9.8),
  (-1.1, 11.5),
  (8.3, 13.1),
  (10.9, -21.7),
  (15.4, 16.8),
  (-17.2, 18.6),
  (19.7, -20.3)])

### Scorer

In [3]:
def rating(category):
    if category == "Excellent":
        return 1
    elif category == "Good":
        return 2
    elif category == "Fair":
        return 3
    elif category == "Poor":
        return 4
    elif category == "Very Poor":
        return 5
    else:
        return 6

def scorer_llm_helper(wb_pair):
    system_prompt = """\
    {xy_pairs}

    Above are the (x, y) pairs representing data points. 
    You are an AI model trained to evaluate the quality of a line defined by parameters w (slope) 
    and b (y-intercept) in terms of how well it fits the set of data points. 
    The equation of a line is y = w * x + b, where w is the slope and b is the y-intercept.
    The quality of the line can be placed into one of 5 categories:
    1. Excellent
    2. Good
    3. Fair
    4. Poor
    5. Very Poor

    ## Excellent
    A line that fits the data points very well and passes through as many points as possible while having the 
    lowest possible error rate between the points and the line should be labelled as "Excellent".

    ## Good
    A line that fits the data points well and passes through many of the points while having a small error rate
    between the points and the line should be labelled as "Good".

    ## Fair
    A line that fits the data points reasonably well and passes through some of the points while having a moderate
    error rate between the points and the line should be labelled as "Fair".

    ## Poor
    A line that fits the data points poorly and passes through few of the points while having a large error rate
    between the points and the line should be labelled as "Poor".

    ## Very Poor
    A line that fits the data points very poorly and passes through none of the points while having an incredibly 
    large error rate between the points and the line should be labelled as "Very Poor".

    
    You will receive a (w, b) pair. Your task is to evaluate the pair and return a json object assigning the 
    (w, b) pair a categiry based on how well the line y = w * x + b fits the data points.

    Example:

    Prompt: (9999, 9999)
    
    Let's think step by step. 

    The line y = 9999 * x + 9999 is a line that fits the data points very poorly and passes through none of the
    (x, y) data points.
    
    To calculate the error rate between the line and the points we need some type of cost function.
    In this case, we could use mean-squared error, mean squared logarithmic error, mean absolute error, 
    huber loss, quantile loss, or any other cost function with respect to all the (x, y) points shared above.
    We get an incredibly large error rate between the points and the line.
    
    The line y = 9999 * x + 9999 is a line that fits the data points very poorly and passes through none of the
    (x, y) data points while having an incredibly large error rate between the points and the line. 
    Therefore, the category for the line y = 9999 * x + 9999 is "Very Poor".

    Response: {{(9999, 9999): "Very Poor"}}

    Prompt: ({w_true}, {b_true})

    Let's think step by step.

    The line y = {w_true} * x + {b_true} is a line that fits the data points very well and passes through as many
    of the (x, y) data points as possible.

    To calculate the error rate between the line and the points we need some type of cost function.
    We get the lowest possible error rate between the points and the line.

    Therefore, the category for the line y = {w_true} * x + {b_true} is "Excellent".

    Response: {{({w_true}, {b_true}): "Excellent"}}
    """
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system", 
                "content": system_prompt.format(xy_pairs=[(round(x[0], 2), round(y[0], 2)) for x, y in zip(X, y)], w_true=w_true, b_true=b_true)
            },
            {
                "role": "user", 
                "content": str(wb_pair)
            },
        ],
        model="gpt-3.5-turbo-0125",
        max_tokens=4096,
        temperature=1,
        response_format={"type": "json_object"}
    )
    result = chat_completion.choices[0].message.content
    result = ast.literal_eval(result)
    result = {ast.literal_eval(k): v for k, v in result.items()}
    return result


def scorer_llm(wb_pairs_lst):
    """
    Returns a dictionary of (w, b) pairs and their corresponding categories.
    """
    result = {}
    for wb_pair in wb_pairs_lst:
        result.update(scorer_llm_helper(wb_pair))
    return result

scorer_llm(tuple_list)

{(-2.5, 3.5): 'Fair',
 (0.8, 0.2): 'Fair',
 (4.6, -7.4): 'Fair',
 (6.2, -9.8): 'Fair',
 (-1.1, 11.5): 'Fair',
 (8.3, 13.1): 'Fair',
 (10.9, -21.7): 'Poor',
 (15.4, 16.8): 'Very Poor',
 (-17.2, 18.6): 'Fair',
 (19.7, -20.3): 'Very Poor'}

### Plotting results

In [4]:
# Generate a list of random pairs of w, b
total_pairs = 5
w = (np.random.rand(total_pairs)* 10).astype(int)
b = (np.random.rand(total_pairs) * 10).astype(int)
wb_pairs_lst = [(w[i], b[i]) for i in range(total_pairs)]
wb_pairs = scorer_llm(wb_pairs_lst)
wb_pairs = dict(sorted(wb_pairs.items(), key=lambda item: rating(item[1])))
wb_pairs

{(2, 5): 'Fair',
 (3, 1): 'Fair',
 (2, 1): 'Fair',
 (4, 8): 'Very Poor',
 (4, 5): 'Very Poor'}

In [5]:
def get_text_score_pairs(wb_pairs):
    # Assuming wb_pairs is sorted in ascending order of their function values
    text_score_pairs = ""
    for i, (k,v) in enumerate(wb_pairs.items()):
        if i == total_pairs:
            break
        text_score_pairs += f"""
        input:
        w={k[0]}, b={k[1]}
        category:
        {v}
        """
    return text_score_pairs

steps = 15
for i in tqdm(range(steps)):
    # Optimizer LLM
    output = opt_llm(get_text_score_pairs(wb_pairs))
    new_wb_pairs = ast.literal_eval(output)
    print(new_wb_pairs)
    
    # Scoring the new pairs and updating the wb_pairs
    wb_pairs.update(scorer_llm(new_wb_pairs))

    # Sorting the wb_pairs
    wb_pairs = dict(sorted(wb_pairs.items(), key=lambda item: rating(item[1])))

wb_pairs

  0%|          | 0/15 [00:00<?, ?it/s]

[(-1.5, -3.7), (-2.3, 5.8), (0.8, -11.2), (3.4, 9.1), (1.7, 6.6), (-4.5, 2.3), (4.9, -7.1), (-3.2, -4.6), (5.6, 8.9), (-5.1, 3.4)]


  7%|▋         | 1/15 [00:15<03:31, 15.13s/it]

[(-1.8, 3.2), (0.7, -1.4), (1.3, -0.9), (3.5, 9.1), (-2.4, 6.5), (4.7, -7.8), (-5.2, -8.6), (6.9, 13.2), (5.1, -7.3), (-3.6, 4.8)]


 13%|█▎        | 2/15 [00:29<03:07, 14.44s/it]

...

[(-2.5, 1.1), (2.7, -4.3), (4.6, 0.8), (-3.9, 5.2), (0.3, -2.4), (3.9, -0.7), (5.7, 2.3), (-5.1, 3.7), (1.6, -1.8), (-4.2, 7.4)]


 60%|██████    | 9/15 [02:06<01:24, 14.00s/it]

[(4.2, 5.6), (-6.7, 7.3), (3.9, -9.2), (0.1, 6.6), (8.8, -1.4), (-5.5, 2.3), (2.4, 4.8), (-3.1, -4.5), (1.7, 8.9), (-7.2, -5.3)]


 60%|██████    | 9/15 [02:23<01:35, 15.97s/it]


ValueError: malformed node or string on line 1: <ast.Name object at 0x7fc279bd3340>

In [ ]:
best_category = list(wb_pairs.values())[0]
best_wb_pairs = dict(filter(lambda x: x[1] == best_category, wb_pairs.items()))

while len(best_wb_pairs) > 1:
    # Scoring the best pairs
    best_wb_pairs = scorer_llm(list(best_wb_pairs.keys()))

    # Sorting the wb_pairs
    best_wb_pairs = dict(sorted(best_wb_pairs.items(), key=lambda item: rating(item[1])))

    # Redefine wb_pairs
    best_wb_pairs = dict(filter(lambda x: x[1] == best_category, wb_pairs.items()))
    print(best_wb_pairs)

best_wb_pairs

In [ ]:
# Plot the data with the best fit line
best_w, best_b = list(best_wb_pairs.keys())[0]
y_pred = best_w * X + best_b
plt.plot(X, y, "b.")
plt.plot(X, y_pred, "r-")
plt.xlabel("$x_1$", fontsize=18)
plt.ylabel("$y$", rotation=0, fontsize=18)
plt.axis([0, 10, 0, 100])
plt.show()

In [ ]:
len(wb_pairs)
print(wb_pairs)